In [1]:
#  Chargement des packages Python
import pandas as pd
import numpy as np
import cv2


In [2]:
# PARAMETRES
PrjLocation = '/home/ubuntu/XRAY/'              # CHemin Projet
ArchiveFolder = 'COVID-19_Radiography_Dataset/' # Dossier Archive


In [4]:
# Récupération des metadata
df_Normal = pd.read_excel(PrjLocation + ArchiveFolder + 'Normal.metadata.xlsx')
df_COVID = pd.read_excel(PrjLocation + ArchiveFolder + 'COVID.metadata.xlsx')
df_LungOpacity = pd.read_excel(PrjLocation + ArchiveFolder + 'Lung_Opacity.metadata.xlsx')
df_ViralPneumonia = pd.read_excel(PrjLocation + ArchiveFolder + 'Viral Pneumonia.metadata.xlsx')


In [5]:
# Ajout feature Class
df_Normal['Class'] = 'Normal'
df_COVID['Class'] = 'COVID'
df_LungOpacity['Class'] = 'Lung Opacity'
df_ViralPneumonia['Class'] = 'Viral Pneumonia'



In [31]:
# Ajout des images Normal
df_Normal['FILE NAME'] = df_Normal['FILE NAME'].str.replace('NORMAL','Normal') # Trick pour obtenir la bonne casse des noms de fichiers
NormalFolder = PrjLocation + ArchiveFolder + 'Normal/'
IMG_NORMAL = []
for file in df_Normal.loc[:8500,'FILE NAME']:
    img = cv2.imread(NormalFolder + 'images/' + file + '.png')
    IMG_NORMAL.append(img)


In [26]:
df_Normal.shape


(10192, 5)